# Определение стоимости автомобилей

Сервис по продаже автомобилей с пробегом «Не бит, не крашен» разрабатывает приложение для привлечения новых клиентов. В нём можно быстро узнать рыночную стоимость своего автомобиля. В вашем распоряжении исторические данные: технические характеристики, комплектации и цены автомобилей. Вам нужно построить модель для определения стоимости. 

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

## Подготовка данных

In [1]:
!pip install lightgbm

In [2]:
!pip install scikit-learn==1.1.3

In [3]:
!pip install catboost

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
import sklearn

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.model_selection import cross_val_score, RepeatedKFold, GridSearchCV

from sklearn.metrics import mean_squared_error

from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from catboost import (
    CatBoostRegressor, 
    cv, 
    Pool
)

import lightgbm
print(lightgbm.__version__)
print(sklearn.__version__)

3.3.5
1.1.3


In [5]:
data = pd.read_csv('/datasets/autos.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/datasets/autos.csv'

### EDA

In [ ]:
data.head()

In [ ]:
data.info()

**Вывод:** 
* В датасете есть пропуски. 
* Переменные, отражающие дату и время, требуют преобразования к соответсвующему типу.

In [ ]:
data.columns = data.columns.str.lower()

In [ ]:
for col in data.columns:
    missing_values = data[col].isna().mean()
    if missing_values > 0:
        print(f'В столбце "{col}" {np.round((missing_values * 100), decimals = 2)} % пропусков')

In [ ]:
data.duplicated().sum()

In [ ]:
data[data.duplicated(keep='last')]
data.drop_duplicates()

In [ ]:
data = data.drop_duplicates()

**Вывод:** 
* Доля пропущенных значений велика и доходит до `20` процентов. 
* Дупликатов всего 4. Они были удалены.
* Пропущенные значения заменим на `'no_info'`. Это будет отдельное значение для переменной.  

In [ ]:
data['repaired'] = data['repaired'].replace('yes', 1)
data['repaired'] = data['repaired'].replace('no', 0)
data['repaired'] = data['repaired'].fillna(-1)

In [ ]:
data = data.fillna('no_info')

In [ ]:
date_time_columns = ['datecrawled', 'datecreated', 'lastseen']

In [ ]:
for col in date_time_columns:
    data[col] = pd.to_datetime(data[col], format='%Y-%m-%d %H:%M:%S')

In [ ]:
data.head()

**Вывод:** переменные с датой и временем преобразованы.

In [ ]:
sns.histplot(data['price'], bins=50)
None

In [ ]:
sns.boxplot(data['price'])
None

In [ ]:
data.head()

In [ ]:
sns.boxplot(data['kilometer'])
None

In [ ]:
sns.boxplot(data['power'])
None

**Вывод:** распределение значение перменной `'price'` скошено вправо. Много значений близких к нулю. Можно предположить, что такие значения оставлены сознательно. Тем не менее, необходимости их исправлять нет. 

In [ ]:
data = data.query('price > 0')
data = data.query('power > 0 and power < 1000')

### Определение признаков:

In [ ]:
data = data.drop(['datecrawled', 'datecreated',	'numberofpictures',	'postalcode','lastseen'], axis=1)

### Разделение данных на выборки и признаки:

In [ ]:
X = data.drop(['price'], axis=1)
y = data['price']

In [ ]:
X.shape, y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, random_state = 42)

In [ ]:
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size = 0.5, random_state = 42)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape, X_valid.shape, y_valid.shape

Определение признаков:

In [ ]:
cat_cols = X_train.select_dtypes(include='object').columns.to_list()

num_cols = [
    'registrationyear',
    'power',
    'kilometer',
    'registrationmonth',
]

In [ ]:
cat_cols

In [ ]:
num_cols

### Преобразование категориальных признаков и масштабирование:

In [ ]:
X_train_ohe = X_train.copy()
X_test_ohe = X_test.copy()
X_valid_ohe = X_valid.copy()

In [ ]:
sc = StandardScaler()
X_train_ohe[num_cols] = sc.fit_transform(X_train_ohe[num_cols])
X_test_ohe[num_cols] = sc.transform(X_test_ohe[num_cols])
X_valid_ohe[num_cols] = sc.transform(X_valid_ohe[num_cols])

In [ ]:
X_train.shape

In [ ]:
%%time
ohe = OneHotEncoder(handle_unknown='ignore', drop='first', sparse=False)
ohe.fit(X_train_ohe[cat_cols])
X_train_ohe[
    ohe.get_feature_names_out()
] = ohe.transform(X_train_ohe[cat_cols])
X_train_ohe = X_train_ohe.drop(cat_cols, axis=1)

X_test_ohe[
    ohe.get_feature_names_out()
] = ohe.transform(X_test_ohe[cat_cols])
X_test_ohe = X_test_ohe.drop(cat_cols, axis=1)

X_valid_ohe[
    ohe.get_feature_names_out()
] = ohe.transform(X_valid_ohe[cat_cols])
X_valid_ohe = X_valid_ohe.drop(cat_cols, axis=1)

### Получение метрик:

In [ ]:
def print_metric(y, y_pred):
    print(f'RMSE: {np.sqrt(mean_squared_error(y, y_pred))}')

**Вывод:** 
* был проведен EDA-анализ, в результате которого были устранены пропуски в данных, преобразованы переменные и удалены дупликаты.
* далее отдельно были выделены категориальные и количественные переменные.
* для разделения данных на выборки было необходимо отбросить все перменные связанные с датой, 'numberofpictures', 'postalcode'.
* для тестовой выборки выделили 40% датасета.
* так как задача проекта - задача регрессии, категориальные признаки было необходимо преобразовать в количественные, а затем их масштабировать.
* основная метрика оценки - RMSE. Для ее получения была написана функция.

## Обучение моделей

### Линейная регрессия

In [ ]:
start_lr = time.time()

lr = LinearRegression()
lr.fit(X_train_ohe, y_train)

end_lr = time.time()
elapced_time_lr = end_lr - start_lr
print(elapced_time_lr)

In [ ]:
start_pred_lr = time.time()
pred_lr = lr.predict(X_valid_ohe)
end_pred_lr = time.time()

elapced_time_lr_pred = end_pred_lr - start_pred_lr 

print(elapced_time_lr_pred)
print_metric(y_valid, pred_lr)

### LightGBM

In [ ]:
cat_X_train = X_train.select_dtypes('object').columns
X_train[cat_X_train] = X_train[cat_X_train].astype('category')

cat_X_test = X_test.select_dtypes("object").columns
X_test[cat_X_test] = X_test[cat_X_test].astype('category')

cat_X_valid = X_valid.select_dtypes('object').columns
X_valid[cat_X_valid] = X_valid[cat_X_valid].astype('category')

In [ ]:
X_train

In [ ]:
parameters = {
    'boosting_type': ["gbdt", "goss"],
    "n_estimators":[400, 500],
    'max_depth': [5, 10],  
}

In [ ]:
%%time
lgbm_reg = LGBMRegressor()
lgbm_reg_CV = GridSearchCV(estimator=lgbm_reg,
                   param_grid=parameters,
                   scoring='neg_root_mean_squared_error',
                   cv=5,
                   verbose=0)
lgbm_reg_CV.fit(X_train, y_train)

In [ ]:
lgbm_reg_CV.best_params_

In [ ]:
start_lgbm_reg = time.time()

lgbm_reg = LGBMRegressor(boosting_type='gbdt', max_depth=10, n_estimators=500)
lgbm_reg.fit(X_train, y_train)

end_lgbm_reg = time.time()
elapced_time_lgbm_reg = end_lgbm_reg - start_lgbm_reg 
print(elapced_time_lgbm_reg)

In [ ]:
start_pred_lgbm_reg = time.time()
pred_lgbm_reg = lgbm_reg.predict(X_valid)
end_pred_lgbm_reg = time.time()

elapced_time_lgbm_reg_pred = end_pred_lgbm_reg - start_pred_lgbm_reg

print_metric(y_valid, pred_lgbm_reg)
print(elapced_time_lgbm_reg_pred)

### CatBoost

In [ ]:
cat_cols = ['vehicletype', 'gearbox', 'model', 'fueltype', 'brand']

In [ ]:
start_cb_update = time.time()

cb_update = CatBoostRegressor(iterations=50, depth=10, loss_function='RMSE')
cb_update.fit(
    X_train,
    y_train,
    cat_features=cat_cols, 
    eval_set=(X_valid, y_valid),
    verbose=False,
    plot=False
)
end_cb_update = time.time()
elapced_time_cb_update = end_cb_update - start_cb_update
print(elapced_time_cb_update)

In [ ]:
start_cb_update_pred = time.time()
pred_cb = cb_update.predict(X_valid)
end_cb_update_pred = time.time()
elapced_time_cb_update_pred = end_cb_update_pred - start_cb_update_pred

print_metric(y_valid, pred_cb)
print(elapced_time_cb_update_pred)

**Вывод:** 
* были обучены три модели: `линейная регрессия`, `LightGBM Regressor` и `CatBoost Regressor`. 
* для улучшения результата была применена кросс-валидация с подбором гиперпараметров.
* при установлении гиперпараметров результаты улучшились.

## Анализ моделей

Визуализируем данные: 

In [ ]:
data_score = pd.DataFrame(np.array([[np.sqrt(mean_squared_error(y_valid, lr.predict(X_valid_ohe))), elapced_time_lr], 
                                  [np.sqrt(mean_squared_error(y_valid, lgbm_reg.predict(X_valid))), elapced_time_lgbm_reg], 
                                  [np.sqrt(mean_squared_error(y_valid, cb_update.predict(X_valid))), elapced_time_cb_update]]),
                        columns=['RMSE', 'Время обучения'],
                        index=['Линейная регрессия', 'LightGBM Regressor', 'CatBoost Regressor']
                       )
data_score.apply(lambda x: np.round(x, decimals = 2))

## Проверка лучшей модели: 

Лучшую метрику показал `LightGBM Regressor`, а время обучения - `CatBoost Regressor`. Проверим модели на тестовой выборке.

In [ ]:
lgbm_test = LGBMRegressor(boosting_type='gbdt', max_depth=10, n_estimators=500)
lgbm_test.fit(X_test, y_test)
pred_lgbm_test = lgbm_test.predict(X_test)
print_metric(y_test, pred_lgbm_test)

In [ ]:
cb_test = CatBoostRegressor(iterations=50, depth=10, loss_function='RMSE')
cb_test.fit(
    X_test,
    y_test,
    cat_features=cat_cols, 
    eval_set=(X_valid, y_valid),
    verbose=False,
    plot=False
)
pred_cb_test = cb_test.predict(X_test)
print_metric(y_test, pred_cb_test)

**Вывод:** лучший результат показал `LightGBM Regressor`

**Вывод:** лучшая метрика у `LightGBM Regressor`

## Заключение

Получив данные мы провели разведывательный анализ в ходе которого были устранены пропуски в данных, преобразованы переменные и удалены дупликаты. Далее отдельно были выделены категориальные и количественные переменные. Отбросили все перменные связанные с датой, 'numberofpictures', 'postalcode'. Затем разделили данные на выборки. Для тестовой выборки выделили 40% датасета. Так как задача проекта - задача регрессии, категориальные признаки было необходимо преобразовать в количественные, а затем их масштабировать. Для упрощения получения основной метрики (RMSE) была написана функция.

В ходе проекта были обучены три модели: `линейная регрессия`, `LightGBM Regressor` и `CatBoost Regressor`. Для улучшения результата применили кросс-валидация с подбором гиперпараметров. С измененными гиперпараметрами оценки улучшились. 

По итогам, быстрее всех обучается Линейная регрессия, самый низкий RMSE у LightGBM Regressor. Однако при самом низком RMSE у LightGBM Regressor самая выокая скорость обучения. Таким образом, наиболее оптимальным является CatBoost Regressor.